# Transfer Learning

In [1]:
import numpy                      as np
import tensorflow                 as tf
from tensorflow.keras.models      import Model
from tensorflow.keras.layers      import Dense, Flatten, Input
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tds
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input


## Data Acquisition

In [2]:
(train_img, train_label), (test_img, test_label) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [3]:
def pre_process(input_img):
    input_img = input_img.astype(float)
    return preprocess_input(input_img)


train_img = pre_process(train_img)
test_img = pre_process(test_img)

In [4]:
test_img.shape

(10000, 32, 32, 3)

## Model

In [5]:
'''
Feature Extraction is performed by ResNet50 pretrained on imagenet weights. 
Input size is 224 x 224.
'''

# instantiate Resnet50
def feature_extract(inp):
  feature_extract = ResNet50(input_shape=(224,224,3), 
                             include_top = False, 
                             weights = 'imagenet')(inp)
  return feature_extract  



def classifier(inp):
  x = tf.keras.layers.GlobalAveragePooling2D()(inp)
  x = Flatten()(x)
  x = Dense(512, activation='relu')(x)
  x = Dense(10, activation='softmax')(x)  
  return x


'''
Since input image size is (32 x 32), first upsample the image by factor of (7x7) to transform it to (224 x 224)
Connect the feature extraction and "classifier" layers to build the model.
'''
def final_model():
  inputs = Input(shape= (32,32,3))
  x = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)
  x = feature_extract(x)
  x = classifier(x)

  model = Model(inputs, x)
  return model


model = final_model()  



94773248/94765736 [==============================] - 1s 0us/step


In [6]:
model.summary()
model.compile(optimizer='SGD', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130  

In [7]:
history = model.fit(
            train_img, train_label,
            validation_data=(test_img, test_label),
            epochs=2,
            verbose=1)

Epoch 1/2
1563/1563 [==============================] - 576s 345ms/step - loss: 0.5712 - accuracy: 0.8081 - val_loss: 0.1819 - val_accuracy: 0.9390
Epoch 2/2
1563/1563 [==============================] - 538s 345ms/step - loss: 0.0964 - accuracy: 0.9679 - val_loss: 0.1456 - val_accuracy: 0.9518
